In [1]:
from bbq.analyse import *
import pandas as pd
from datetime import datetime, timedelta
from talib.abstract import *
import talib
import mplfinance as mpf
from bbq.data.stockdb import StockDB
from bbq.data.funddb import FundDB
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import json

db_stock = StockDB()
db_fund = FundDB()
db_stock.init()
db_fund.init()

data = await db_stock.load_stock_daily(filter={'code': 'sz002132'}, limit=60, sort=[('trade_date', -1)])
data = data[::-1]

name = await db_stock.load_stock_info(filter={'code': 'sz002132'}, projection=['name'])
data['name'] = name.iloc[0]['name']


data_fund = await db_fund.load_fund_daily(filter={'code': '159949'}, limit=60, sort=[('trade_date', -1)])
data_fund = data_fund[::-1]

name_fund = await db_fund.load_fund_info(filter={'code': '159949'}, projection=['short_name'])
data_fund['name'] = name_fund.iloc[0]['short_name']

2020-12-29 20:30:02,681 - StockDB - DEBUG - 加载股票日线, kwargs={'filter': {'code': 'sz002132'}, 'limit': 60, 'sort': [('trade_date', -1)]}
2020-12-29 20:30:02,875 - StockDB - DEBUG - 加载日线数据成功 size=60
2020-12-29 20:30:02,879 - StockDB - DEBUG - 加载股票信息, kwargs={'filter': {'code': 'sz002132'}, 'projection': ['name']} ...
2020-12-29 20:30:02,957 - StockDB - DEBUG - 加载股票信息成功 size=1
2020-12-29 20:30:02,967 - FundDB - DEBUG - 加载场内基金日线数据, kwargs={'filter': {'code': '159949'}, 'limit': 60, 'sort': [('trade_date', -1)]}
2020-12-29 20:30:04,103 - FundDB - DEBUG - 加载场内基金日线数据,成功, size=60
2020-12-29 20:30:04,105 - FundDB - DEBUG - 加载基金列表, kwargs={'filter': {'code': '159949'}, 'projection': ['short_name']}
2020-12-29 20:30:04,458 - FundDB - DEBUG - 加载基金列表成功, size=1


In [26]:
# data = data_fund

k_data = list(zip(data['open'], data['close'], data['low'], data['high']))
trade_date = [d.strftime('%Y-%m-%d') for d in data['trade_date']]


def kline_tooltip_fmt_func():
    return JsCode("""function(obj){
                function tips_str(pre, now, tag, field, unit) {
                    var tips = tag + ':&nbsp;&nbsp;&nbsp;&nbsp;';
                    var span = '';
                    
                    if (field == 'open' || field == 'close' ||
                        field == 'low' || field == 'high') {
                        var fixed = 3;
                        if (now['code'].startsWith('s')) {
                            fixed = 2;
                        }
                        if (pre != null) {
                            var chg = (now[field] - pre['close']).toFixed(fixed);
                            var rate = (chg * 100 / pre['close']).toFixed(2);

                            if (rate >= 0) {
                                span = '<span style="color: #F11300;">' + now[field].toFixed(fixed) + '&nbsp;&nbsp;(' + rate + '%,&nbsp;' + chg + ')</span><br>';
                            } else {
                                span = '<span style="color: #00A800;">' + now[field].toFixed(fixed) + '&nbsp;&nbsp;(' + rate + '%,&nbsp;' + chg + ')</span><br>';
                            }

                        } else {
                            span = now[field].toFixed(fixed) + '&nbsp;&nbsp;(' + rate + '%,&nbsp;' + rate + ')<br>';
                        }
                    } else {
                        if (field == 'volume') {
                            span = (now[field] / 1000000.0).toFixed(2) + '&nbsp;万手<br>';
                        }
                        if (field == 'turnover') {
                            span = now[field] + '%<br>'
                        }
                    }
                    return tips + span;

                }
                
                var pre_data = null;
                var now_data = kdata[trade_date[obj.dataIndex]];
                if (obj.dataIndex > 0) {
                    pre_data = kdata[trade_date[obj.dataIndex-1]];
                }
                var title = now_data['code'] + '&nbsp;&nbsp;' + trade_date[obj.dataIndex] + '<br><br>';

                if ('name' in now_data) {
                    title = now_data['name'] + '(' + now_data['code'] + ')&nbsp;&nbsp;' + trade_date[obj.dataIndex] + '</div><br>';
                }
                
                return title + 
                    tips_str(pre_data, now_data, '开盘', 'open') + 
                    tips_str(pre_data, now_data, '最高', 'high') + 
                    tips_str(pre_data, now_data, '最低', 'low') + 
                    tips_str(pre_data, now_data, '收盘', 'close') + 
                    tips_str(pre_data, now_data, '成交', 'volume') + 
                    tips_str(pre_data, now_data, '换手', 'turnover');
                 }""")

def kline_orig_data(data):
    origData = data[:]
    origData['trade_date'] = origData['trade_date'].apply(lambda d: d.strftime('%Y-%m-%d'))
    origData.index = origData['trade_date']
    origData.fillna('-', inplace=True)
    return origData.to_dict('index')

kline = Kline()
#kline.add_js_funcs('var kdata={}'.format(kline_orig_data(data)))
#kline.add_js_funcs('var trade_date={}'.format(trade_date))
kline.add_xaxis(trade_date)
kline.add_yaxis('日线', k_data,
                itemstyle_opts=opts.ItemStyleOpts(
                    color='#F11300',
                    color0='#00A800',
                ),
#                markline_opts=opts.MarkLineOpts(
#                    data=[opts.MarkLineItem(type_="max", value_dim="highest")])
                tooltip_opts=opts.TooltipOpts(
                    formatter=kline_tooltip_fmt_func())
               )
kline.set_global_opts(xaxis_opts=opts.AxisOpts(is_scale=True),
                     yaxis_opts=opts.AxisOpts(
                         is_scale=True,
                         splitarea_opts=opts.SplitAreaOpts(
                             is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                         ),
                     ),
                      datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%", filter_mode='none')],
                      title_opts=opts.TitleOpts(title="Kline-MarkLine"))

ma5 = MA(data['close'], timeperiod=5)
ma5 = [round(v, 3) for v in ma5]

ma10 = MA(data['close'], timeperiod=10)
ma10 = [round(v, 3) for v in ma10]

wma = WMA(data['close'], timeperiod=2)

line = Line()
line.add_xaxis(trade_date)
line.add_yaxis('ma5', ma5, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')
line.add_yaxis('ma10', ma10, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')
line.add_yaxis('wma', data['close'], label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')

kline.overlap(line)
#scatter = Scatter()
#scatter.add_xaxis(trade_date)
#scatter.add_yaxis('买入', [data.iloc[30]['high']], )
#line.render_notebook()

bar = Bar()
bar.add_xaxis(xaxis_data=trade_date)
bar.add_yaxis(
            "Volumn", list(data["volume"]),
            xaxis_index=1,
            yaxis_index=1,
            label_opts=opts.LabelOpts(is_show=False),
            # 根据 echarts demo 的原版是这么写的
            # itemstyle_opts=opts.ItemStyleOpts(
            #     color=JsCode("""
            #     function(params) {
            #         var colorList;
            #         if (data.datas[params.dataIndex][1]>data.datas[params.dataIndex][0]) {
            #           colorList = '#ef232a';
            #         } else {
            #           colorList = '#14b143';
            #         }
            #         return colorList;
            #     }
            #     """)
            # )
            # 改进后在 grid 中 add_js_funcs 后变成如下
            itemstyle_opts=opts.ItemStyleOpts(
                color=JsCode(
                    """
                function(params) {
                console.log(params);
                    var colorList;
                    if (params.dataIndex == 0) {
                        return '#F11300';
                    }
                    if (barData[params.dataIndex] > barData[params.dataIndex - 1]) {
                        colorList = '#F11300';
                    } else {
                        colorList = '#00A800';
                    }
                    console.log(colorList);
                    return colorList;
                }
                """
                )
            ),
        )

grid_chart = Grid()
grid_chart.add_js_funcs('var barData={}'.format(list(data["volume"])))
grid_chart.add_js_funcs('var kdata={}'.format(kline_orig_data(data)))
grid_chart.add_js_funcs('var trade_date={}'.format(trade_date))

grid_chart.add(
        kline,
        grid_opts=opts.GridOpts(pos_left="3%", pos_right="1%", height="60%"),
    )
grid_chart.add(
        bar,
        grid_opts=opts.GridOpts(
            pos_left="3%", pos_right="1%", pos_top="71%", height="20%"
        ),
    )
grid_chart.render_notebook()
#

In [27]:
data

,code,trade_date,close,open,high,low,volume,turn_over,hfq_factor,name
59,sz002132,2020-09-28,3.22,2.91,3.22,2.91,36362431.0,NaN,5.8647257141694000,恒星科技
58,sz002132,2020-09-29,3.54,3.54,3.54,3.54,15832260.0,NaN,5.8647257141694000,恒星科技
57,sz002132,2020-09-30,3.89,3.60,3.89,3.56,131102745.0,NaN,5.8647257141694000,恒星科技
56,sz002132,2020-10-09,4.28,4.28,4.28,4.28,7945992.0,NaN,5.8647257141694000,恒星科技
55,sz002132,2020-10-12,4.71,4.71,4.71,4.71,11224327.0,NaN,5.8647257141694000,恒星科技
54,sz002132,2020-10-13,5.18,5.18,5.18,5.18,13492403.0,NaN,5.8647257141694000,恒星科技
53,sz002132,2020-10-14,5.28,5.70,5.70,4.99,556864254.0,NaN,5.8647257141694000,恒星科技
52,sz002132,2020-10-15,4.75,4.75,4.75,4.75,31706400.0,NaN,5.8647257141694000,恒星科技
51,sz002132,2020-10-16,4.29,4.40,4.52,4.28,402250793.0,NaN,5.8647257141694000,恒星科技
50,sz002132,2020-10-19,4.24,4.17,4.33,4.12,220863966.0,NaN,5.8647257141694000,恒星科技
